### SocialAL Models
# Data simulation and parameter recovery - multiple subjects
KLS 8.30.19; update 7.7.22  
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version

In [1]:
import sys
print(sys.version)  

3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import beta
from scipy.stats import gamma
from scipy.stats import uniform
import matplotlib.pyplot as plt
import itertools

## Pull in functions

In [3]:
run common_functions.ipynb

In [4]:
run priors_functions.ipynb

## Functions to simulate data

In [5]:
def select_response(prob):     
    n = random.uniform(0,1)
    if n > prob:
        response = 0
    else:
        response = 1
    return response

### Set constants

In [6]:
recip_rates = {0: 0.93, 1:0.6, 2:0.07}

### New function to simulate data for one sub

In [7]:
def sim_data(tn, params):
    # tn: number of trials desired
    # params: ground truth of parameters
    a_gain = params[0]
    a_loss = params[1]
    beta = params[2]
    prob = params[3:6]
    
    # initialize variables
    Probs = [0.5, 0.5, 0.5]
    EVs = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    
    # generate trial sequence
    trial_sequence = np.repeat([0,1,2], tn)
    random.shuffle(trial_sequence) #print(trial_sequence)
    
    trial = []
    choices = []
    responses = []
    
    for x in range(0,len(trial_sequence)):
        t = trial_sequence[x] 
        
        # Trial
        trial.append(x+1)
        
        # Make a choice
        choice = action_selection(get_action_selection_probs(beta, EVs[t]))
        choices.append(choice) 
    
        # Get a response
        recip_rate = recip_rates.get(t) 
        
        response = select_response(recip_rate) 
        responses.append(response)
    
        # after choice, update probability
        if choice != 1:
            Probs[t] = update_prob(response, Probs[t], a_gain, a_loss) 
        # then update value
        EVs[t] = update_value(Probs[t]) 

    data = {'Trial': trial, 'Stim_Sequence': trial_sequence, 'Choice' : choices, 'Trustee_Response': responses}    
    return data

In [8]:
#sim_data(15,[.2,.3,2,.5, .8, .2])

### Create param space

In [9]:
n = 150
a_gains = beta.rvs(a = 1.1, b = 1.1, size = n)
a_losses = beta.rvs(a = 1.1, b = 1.1, size = n)
betas = gamma.rvs(a = 1.2, scale = 5, size = n)
iprobA = uniform.rvs(size = n)
iprobB = uniform.rvs(size = n)
iprobC = uniform.rvs(size = n)

### Simulate data

In [10]:
data = pd.DataFrame(columns= ['Trial','Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject', 
                              'Alpha_gain', 'Alpha_loss','Beta', 'iProbA', 'iProbB', 'iProbC'])
for p in range(len(betas)):
    dt = sim_data(15, [a_gains[p], a_losses[p], betas[p], iprobA[p], iprobB[p], iprobC[p]])
    dt['Subject'] = [p + 1] * 45
    dt['Alpha_gain'] = a_gains[p] 
    dt['Alpha_loss'] = a_losses[p]
    dt['Beta'] = betas[p]
    dt['iProbA'] = iprobA[p]
    dt['iProbB'] =  iprobB[p]
    dt['iProbC'] = iprobC[p]
    dt = pd.DataFrame(dt)
    data = pd.concat([data, dt])
data

data.to_csv(path_or_buf = '../../output/simulation/sim_2alpha_with_priors_model_data.csv', index = False)

### New function to fit model to multiple subjects

In [11]:
def model_fit_mult_subject(data):
    pdt = pd.DataFrame(columns = ['Subject', 'a_gain', 'a_loss', 'beta', 'iProbA', 'iProbB', 'iProbC'])
    a = pd.unique(data['Subject'])
    for sub in range(1,len(a)+1):
        print('Subject: ', sub)
        df = data[data['Subject']==sub]
        dt = df.to_dict()
        params = model_fit(dt)
        #print('Params: ', params)
        line = {'Subject': sub, 'a_gain': params.x[0], 'a_loss': params.x[1], 'beta':params.x[2], 
                'iProbA':params.x[3], 'iProbB':params.x[4], 'iProbC':params.x[5]}
        pdt = pdt.append(line, ignore_index=True)   
    return(pdt)


In [12]:
precover = model_fit_mult_subject(data)

Subject:  1
Subject:  2
Subject:  3
Subject:  4
Subject:  5
Subject:  6
Subject:  7
Subject:  8
Subject:  9
Subject:  10
Subject:  11
Subject:  12
Subject:  13
Subject:  14
Subject:  15
Subject:  16
Subject:  17
Subject:  18
Subject:  19
Subject:  20
Subject:  21
Subject:  22
Subject:  23
Subject:  24
Subject:  25
Subject:  26
Subject:  27
Subject:  28
Subject:  29
Subject:  30
Subject:  31
Subject:  32
Subject:  33
Subject:  34
Subject:  35
Subject:  36
Subject:  37
Subject:  38
Subject:  39
Subject:  40
Subject:  41
Subject:  42
Subject:  43
Subject:  44
Subject:  45
Subject:  46
Subject:  47
Subject:  48
Subject:  49
Subject:  50
Subject:  51
Subject:  52
Subject:  53
Subject:  54
Subject:  55
Subject:  56
Subject:  57
Subject:  58
Subject:  59
Subject:  60
Subject:  61
Subject:  62
Subject:  63
Subject:  64
Subject:  65
Subject:  66
Subject:  67
Subject:  68
Subject:  69
Subject:  70
Subject:  71
Subject:  72
Subject:  73
Subject:  74
Subject:  75
Subject:  76
Subject:  77
Subject:

In [13]:
print(precover)

     Subject    a_gain    a_loss      beta    iProbA    iProbB        iProbC
0        1.0  0.399974  0.016708  0.717467  1.000000  0.967659  3.108200e-01
1        2.0  0.183169  0.175354  0.786023  0.982873  0.730322  3.633818e-01
2        3.0  0.883858  0.575956  0.735766  0.071290  0.423661  5.376246e-02
3        4.0  0.101933  0.894415  0.763406  0.981018  0.447260  1.057424e-03
4        5.0  0.433945  0.032398  0.406314  0.977758  0.219832  1.721306e-01
5        6.0  0.339935  0.584057  0.678062  0.914326  0.102547  2.207902e-01
6        7.0  0.304591  0.256960  0.925429  0.858450  0.231195  4.917106e-01
7        8.0  0.605948  0.575934  0.901035  0.997779  0.060357  3.203206e-01
8        9.0  0.686513  0.139550  0.109666  0.985290  0.954674  4.263039e-01
9       10.0  0.927101  0.560670  0.769910  0.438016  0.126796  3.630161e-02
10      11.0  0.351146  0.391556  0.799789  0.998494  0.117896  2.155882e-01
11      12.0  0.401005  0.274463  0.744512  0.999578  0.775455  6.914020e-01

In [14]:
precover.to_csv(path_or_buf = '../../output/simulation/sim_2alpha_with_priors_model_fit.csv', index = False)